In [ ]:
import cv2

In [16]:
eyeCascade = cv2.CascadeClassifier("haarcascade_eye.xml")
leyeCascade = cv2.CascadeClassifier("haarcascade_lefteye_2splits.xml")
reyeCascade = cv2.CascadeClassifier("haarcascade_righteye_2splits.xml")

In [18]:
faceCascade= cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [5]:
model = load_model('model-best')

In [6]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 98, 98, 200)       2000      
_________________________________________________________________
activation_15 (Activation)   (None, 98, 98, 200)       0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 47, 47, 100)       180100    
_________________________________________________________________
activation_16 (Activation)   (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 23, 23, 100)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 23, 23, 100)      

In [7]:
model1 = load_model('yaw-yaw')

In [8]:
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 98, 98, 200)       2000      
_________________________________________________________________
activation_10 (Activation)   (None, 98, 98, 200)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 47, 47, 100)       180100    
_________________________________________________________________
activation_11 (Activation)   (None, 47, 47, 100)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 23, 23, 100)       0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 23, 23, 100)      

In [21]:
from time import time

In [49]:
def alarmer():  # Alarm triggered every time fatigued user detected.
    ti1 = time()
    mixer.init()
    mixer.music.load("alarm1.mp3")
    mixer.music.set_volume(0.7)
    mixer.music.play()
    return ti1

In [50]:
cap=cv2.VideoCapture(0)
cap.set(10,100)
blink_count = 0
blink_status = 0
t1 = t1_ = time()
wake = 1
sleep = 0
yawn_count = 0
yawn_status = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray,1.1,4)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        gray_face=imgGray[y:y+h,x:x+w]
        color_face=img[y:y+h,x:x+w]
        resized_image = cv2.resize(gray_face, (100, 100))
        reshaped_face = np.reshape(resized_image, (1, 100, 100, 1))
        predyawn = model1.predict(reshaped_face)
        if predyawn[0][0] < predyawn[0][1]:
            if yawn_status == 0:
                yawn_count += 1
                yawn_status = 1
        else: 
            yawn_status = 0
        
        eye = eyeCascade.detectMultiScale(gray_face,1.3,5)
        leye = leyeCascade.detectMultiScale(gray_face,1.3,5)
        reye = reyeCascade.detectMultiScale(gray_face,1.3,5)
        
        for (a,b,c,d) in leye:
            cv2.rectangle(color_face,(a,b),(a+c,b+d),(0,255,0),2)
            l_eye = color_face[b:b+d,a:a+c]
            resized_image = cv2.resize(l_eye, (100, 100))
            l_eye = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
            #normalized_image = l_eye/255
            reshaped_eye = np.reshape(l_eye, (1, 100, 100, 1))
            lpred = model.predict(reshaped_eye)
                     
        for (a,b,c,d) in reye:
            cv2.rectangle(color_face,(a,b),(a+c,b+d),(0,255,0),2)
            r_eye = color_face[b:b+d,a:a+c]
            resized_image = cv2.resize(r_eye, (100, 100))
            r_eye = cv2.cvtColor(resized_image,cv2.COLOR_BGR2GRAY)
            #normalized_image = l_eye/255
            reshaped_eye = np.reshape(r_eye, (1, 100, 100, 1))
            rpred = model.predict(reshaped_eye)
            
        
        if rpred[0][0] < rpred[0][1] and lpred[0][0] < lpred[0][1]:
            if blink_status == 0:
                blink_count += 1
                blink_status = 1
        else: 
            blink_status = 0
 
 #Eye blink frequency 
    t2 = time()
    count_down = t2 - t1
    #10 count = 4,   count n= 1
    if count_down >= 15:
        if blink_count > 4:
            wake = 1
        else:
            wake = 0     
        t1 = time()
        blink_count = 0
        
    elapsed_time = t2 - t1_ 
    if elapsed_time >= 10:
        if yawn_count > 1:
            sleep = 1
        elif yawn_count == 1:
            sleep = 0.5
        else:
            sleep = 0
        t1_ = time()
        yawn_count = 0    
     

        
    cv2.putText(img,'Yawns = ' + str(yawn_count) + ', Blinks = ' + str(blink_count),(x, y - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    if sleep == 1: 
        from pygame import mixer
        ti1 = alarmer()
        cv2.putText(img,"Fatigued",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        ti2 = time()
        if ti2 - ti1 >= 15:
            mixer.music.stop()
    elif not wake and sleep == 0.5:
        cv2.putText(img,"Semi Fatigued",(x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (57, 143, 235), 2)
    elif wake and sleep == 0.5:
        cv2.putText(img,"Drowsy",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (58, 238, 238), 2)
    elif wake:
        cv2.putText(img,"Awake",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (67, 146, 52), 2)
    else:
        cv2.putText(img,"Semi Drowsy",(x+50, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (57, 235, 143), 2)
   
            
    cv2.imshow("Result", img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
mixer.music.stop()
cap.release()
cv2.destroyAllWindows()

